# Build an App for diarized transcription Using Aana SDK

This notebook provides an example of getting diarized transcription from video. Please note that the pyannote diarization model is a gated model. Follow [speaker diarization deployment docs](./../docs/pages/model_hub/speaker_recognition.md) to get access to the model.

As a first step, set the environment and install aana SDK

In [16]:
import os

os.environ["HF_TOKEN"] = "YOUR_HF_TOKEN_GOES_HERE"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Define Whisper and PyannoteSpeakerDiarization deployments, define the TranscribeVideoWithDiarEndpoint for diarized transcription. Register deployments and the endpoints.

In [11]:
from aana.api.api_generation import Endpoint
from aana.core.models.base import pydantic_to_dict
from aana.core.models.speaker import PyannoteSpeakerDiarizationParams
from aana.core.models.video import VideoInput
from aana.core.models.whisper import WhisperParams
from aana.deployments.aana_deployment_handle import AanaDeploymentHandle
from aana.deployments.pyannote_speaker_diarization_deployment import (
    PyannoteSpeakerDiarizationConfig,
    PyannoteSpeakerDiarizationDeployment,
)
from aana.deployments.whisper_deployment import (
    WhisperComputeType,
    WhisperConfig,
    WhisperDeployment,
    WhisperModelSize,
    WhisperOutput,
)
from aana.integrations.external.yt_dlp import download_video
from aana.processors.remote import run_remote
from aana.processors.speaker import asr_postprocessing_for_diarization
from aana.processors.video import extract_audio
from aana.sdk import AanaSDK

# Define the model deployments.
asr_deployment = WhisperDeployment.options(
    num_replicas=1,
    ray_actor_options={
        "num_gpus": 0.25
    },  # Remove this line if you want to run Whisper on a CPU.# Also change type to float32.
    user_config=WhisperConfig(
        model_size=WhisperModelSize.SMALL,
        compute_type=WhisperComputeType.FLOAT16,
    ).model_dump(mode="json"),
)
diarization_deployment = PyannoteSpeakerDiarizationDeployment.options(
    num_replicas=1,
    ray_actor_options={
        "num_gpus": 0.1
    },  # Remove this line if you want to run the model on a CPU.
    user_config=PyannoteSpeakerDiarizationConfig(
        model_id="pyannote/speaker-diarization-3.1"
    ).model_dump(mode="json"),
)
deployments = [
    {"name": "asr_deployment", "instance": asr_deployment},
    {"name": "diarization_deployment", "instance": diarization_deployment},
]


# Define the endpoint to transcribe the video with diarization.
class TranscribeVideoWithDiarEndpoint(Endpoint):
    """Transcribe video with diarization endpoint."""

    async def initialize(self):
        """Initialize the endpoint."""
        self.asr_handle = await AanaDeploymentHandle.create("asr_deployment")
        self.diar_handle = await AanaDeploymentHandle.create("diarization_deployment")
        await super().initialize()

    async def run(
        self,
        video: VideoInput,
        whisper_params: WhisperParams,
        diar_params: PyannoteSpeakerDiarizationParams,
    ) -> WhisperOutput:
        """Transcribe video with diarization."""
        video_obj = await run_remote(download_video)(video_input=video)
        audio = extract_audio(video=video_obj)

        # diarized transcript requires word_timestamps from ASR
        whisper_params.word_timestamps = True
        transcription = await self.asr_handle.transcribe(
            audio=audio, params=whisper_params
        )
        diarized_output = await self.diar_handle.diarize(
            audio=audio, params=diar_params
        )
        transcription = asr_postprocessing_for_diarization(
            diarized_output, transcription
        )
        output = pydantic_to_dict(transcription)

        output_keys = ["time_interval", "speaker", "text"]
        filtered_data = [
            {k: v for k, v in entry.items() if k in output_keys}
            for entry in output["segments"]
        ]

        return {"segments": filtered_data}


endpoints = [
    {
        "name": "transcribe_video",
        "path": "/video/transcribe",
        "summary": "Transcribe a video",
        "endpoint_cls": TranscribeVideoWithDiarEndpoint,
    },
]

aana_app = AanaSDK(name="transcribe_video_app")

for deployment in deployments:
    aana_app.register_deployment(**deployment)

for endpoint in endpoints:
    aana_app.register_endpoint(**endpoint)

aana_app.connect(
    host="127.0.0.1", port=8000, show_logs=False
)  # Connects to the Ray cluster or starts a new one.
aana_app.migrate()  # Runs the migrations to create the database tables.

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Start the App!

In [12]:
aana_app.deploy()

Deployed successfully.

Documentation is available at ]8;id=337020;http://127.0.0.1:8000/docs\http://127.0.0.1:8000/docs]8;;\ and ]8;id=418863;http://127.0.0.1:8000/redoc\http://127.0.0.1:8000/redoc]8;;\

Now that we have the app running, lets provide an example audio with multiple speakers for transcription.

In [15]:
import json

import requests

video = {
    "path": "../aana/tests/files/audios/sd_sample.wav",  # Video URL, Aana SDK supports URLs (including YouTube), file paths or even raw video data
    "media_id": "sd_sample",  # Media ID, so we can ask questions about the video later by using this ID
}

data = {
    "whisper_params": {
        "word_timestamps": True,  # Enable word_timestamps
    },
    "video": video,
}

url = "http://127.0.0.1:8000/video/transcribe"

# No streaming support possible for diarized transcription as it needs complete ASR output beforehand.
response = requests.post(url, data={"body": json.dumps(data)})

print(response.json())

{'segments': [{'text': ' Hello. Hello.', 'time_interval': {'start': 6.38, 'end': 7.84}, 'speaker': 'SPEAKER_01'}, {'text': " Oh, hello. I didn't know you were there.", 'time_interval': {'start': 8.3, 'end': 9.68}, 'speaker': 'SPEAKER_02'}, {'text': ' Neither did I.', 'time_interval': {'start': 9.98, 'end': 10.64}, 'speaker': 'SPEAKER_01'}, {'text': ' Okay, I thought, you know, I heard it deep. This is Diane in New Jersey.', 'time_interval': {'start': 10.64, 'end': 13.94}, 'speaker': 'SPEAKER_02'}, {'text': " And I'm Sheila in Texas, originally from Chicago.", 'time_interval': {'start': 14.36, 'end': 17.42}, 'speaker': 'SPEAKER_00'}, {'text': " Oh, I'm originally from Chicago also. I'm in New Jersey now, though.", 'time_interval': {'start': 18.04, 'end': 21.34}, 'speaker': 'SPEAKER_02'}, {'text': ' Well, there is not much difference. At least, you know, they all call me a Yankee down here, so what can I say?', 'time_interval': {'start': 21.74, 'end': 28.32}, 'speaker': 'SPEAKER_00'}, {'

Each transcribed segment comes with a corresponding speaker label as well!